Sample ORES Scoring
===

Get ORES scores for the sampled data.

In [1]:
import mwapi
import mwxml
import mwxml.utilities
import mwcli
import mwreverts
import oresapi
import mwparserfromhell

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [3]:
import os
from tqdm import tqdm
import bz2
import gzip
import json
import re
import hashlib
from datetime import datetime
import nltk
import scipy.stats
import para
from itertools import groupby
from collections import Counter

In [4]:
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = git_root_dir[0]
git_root_dir

'/export/scratch2/levon003/repos/wiki-ores-feedback'

In [5]:
raw_data_dir = "/export/scratch2/wiki_data"
derived_data_dir = os.path.join(git_root_dir, "data", "derived")
raw_data_dir, derived_data_dir

('/export/scratch2/wiki_data',
 '/export/scratch2/levon003/repos/wiki-ores-feedback/data/derived')

In [6]:
working_dir = os.path.join(derived_data_dir, 'revision_sample')
os.makedirs(working_dir, exist_ok=True)
working_dir

'/export/scratch2/levon003/repos/wiki-ores-feedback/data/derived/revision_sample'

In [7]:
start_date = datetime.fromisoformat('2018-01-01')
start_timestamp = int(start_date.timestamp())
end_date = datetime.fromisoformat('2020-01-01')
end_timestamp = int(end_date.timestamp())
start_timestamp, end_timestamp

(1514786400, 1577858400)

In [8]:
sample_start_timestamp = start_timestamp
sample_end_date = datetime.fromisoformat('2019-01-01')
sample_end_timestamp = int(end_date.timestamp())

### Load the sample

In [9]:
# read in the sample dataframe
revision_sample_dir = os.path.join(derived_data_dir, 'revision_sample')
sample1_filepath = os.path.join(revision_sample_dir, 'sample1_1M.pkl')
rev_df = pd.read_pickle(sample1_filepath)
len(rev_df)

1000000

In [10]:
rev_df.head()

,page_id,rev_id,rev_timestamp,is_revert_target,is_reverted,is_reverting,is_sample_eligible,prev_rev_id,prev_rev_timestamp,reverted_rev_ids,reverting_rev_id,reverting_rev_timestamp
0,9516095,846835190,1529576038,0,0,0,True,846835009,1529575914,[],-1,-1
1,876872,923997118,1572595746,0,0,0,True,923211396,1572162918,[],-1,-1
2,53465104,867368228,1541425108,0,0,0,True,867366487,1541423455,[],-1,-1
3,60096152,888337768,1552936050,0,0,0,True,888337457,1552935890,[],-1,-1
4,28063274,917010365,1569117229,0,0,0,True,915190631,1568247118,[],-1,-1


### Retrieve ORES predictions

In [12]:
rev_id_list = rev_df.rev_id
len(rev_id_list)

1000000

In [21]:
session = oresapi.Session("https://ores.wikimedia.org", 
                          user_agent="levon003@umn.edu - ores.api for UMN research",
                         parallel_requests=2, batch_size=50)
scores = session.score("enwiki", ["damaging", "goodfaith"], rev_id_list)

In [22]:
scores_list = []
ores_score_filepath = os.path.join(working_dir, 'sample1_ores_scores.csv')
with open(ores_score_filepath, 'w') as outfile:
    for rev_id, score in tqdm(zip(rev_id_list, scores), total=len(rev_id_list)):
        score['rev_id'] = rev_id
        scores_list.append(score)
        damaging_score = -1
        damaging_prediction = ""
        if 'damaging' in score and 'score' in score['damaging']:
            damaging_score = score['damaging']['score']['probability']['true']
            damaging_prediction = str(score['damaging']['score']['prediction'])
        goodfaith_score = -1
        goodfaith_prediction = ""
        if 'goodfaith' in score and 'score' in score['goodfaith']:
            goodfaith_score = score['goodfaith']['score']['probability']['true']
            goodfaith_prediction = str(score['goodfaith']['score']['prediction'])
        outfile.write(f"{rev_id},{damaging_score},{damaging_prediction},{goodfaith_score},{goodfaith_prediction}\n")



 26%|██▌       | 259651/1000000 [6:35:45<18:35:06, 11.07it/s]

 26%|██▌       | 259751/1000000 [6:35:56<19:47:03, 10.39it/s]

 26%|██▌       | 259851/1000000 [6:35:59<15:59:12, 12.86it/s]

 26%|██▌       | 259901/1000000 [6:36:02<15:04:32, 13.64it/s]

 26%|██▌       | 259951/1000000 [6:36:13<23:44:53,  8.66it/s]

 26%|██▌       | 260051/1000000 [6:36:22<22:14:16,  9.24it/s]

 26%|██▌       | 260101/1000000 [6:36:26<20:12:02, 10.17it/s]

 26%|██▌       | 260151/1000000 [6:36:33<22:17:37,  9.22it/s]

 26%|██▌       | 260201/1000000 [6:36:38<21:39:10,  9.49it/s]

 26%|██▌       | 260251/1000000 [6:36:41<19:24:16, 10.59it/s]

 26%|██▌       | 260301/1000000 [6:36:46<19:12:09, 10.70it/s]

 26%|██▌       | 260351/1000000 [6:36:50<18:43:32, 10.97it/s]

 26%|██▌       | 260401/1000000 [6:36:54<17:50:05, 11.52it/s]

 26%|██▌       | 260451/1000000 [6:36:58<18:06:22, 11.35it/s]

 26%|██▌       | 260501/1000000 [6:37:06<22:18:36,  9.21it/s]

 26%|██▌       | 260601/1000000 [6:37:19<23:23:31,  8

 27%|██▋       | 273751/1000000 [6:57:01<16:53:43, 11.94it/s]

 27%|██▋       | 273801/1000000 [6:57:04<14:39:36, 13.76it/s]

 27%|██▋       | 273851/1000000 [6:57:07<14:06:16, 14.30it/s]

 27%|██▋       | 273901/1000000 [6:57:16<21:26:14,  9.41it/s]

 27%|██▋       | 273951/1000000 [6:57:20<18:51:13, 10.70it/s]

 27%|██▋       | 274001/1000000 [6:57:24<18:08:06, 11.12it/s]

 27%|██▋       | 274051/1000000 [6:57:29<19:15:57, 10.47it/s]

 27%|██▋       | 274101/1000000 [6:57:31<15:36:32, 12.92it/s]

 27%|██▋       | 274151/1000000 [6:57:35<16:27:31, 12.25it/s]

 27%|██▋       | 274201/1000000 [6:57:40<16:36:05, 12.14it/s]

 27%|██▋       | 274251/1000000 [6:57:46<19:49:57, 10.16it/s]

 27%|██▋       | 274301/1000000 [6:57:47<14:56:52, 13.49it/s]

 27%|██▋       | 274351/1000000 [6:57:55<19:14:11, 10.48it/s]

 27%|██▋       | 274401/1000000 [6:57:58<17:24:26, 11.58it/s]

 27%|██▋       | 274451/1000000 [6:58:01<16:06:26, 12.51it/s]

 27%|██▋       | 274501/1000000 [6:58:14<26:35:19,  7.5

 29%|██▊       | 287351/1000000 [7:17:01<16:40:33, 11.87it/s]

 29%|██▊       | 287451/1000000 [7:17:05<21:30:08,  9.21it/s]

 29%|██▉       | 287601/1000000 [7:17:16<19:26:35, 10.18it/s]

 29%|██▉       | 287651/1000000 [7:17:17<14:53:45, 13.28it/s]

 29%|██▉       | 287701/1000000 [7:17:28<23:07:43,  8.55it/s]

 29%|██▉       | 287801/1000000 [7:17:32<18:41:33, 10.58it/s]

 29%|██▉       | 287851/1000000 [7:17:35<17:19:02, 11.42it/s]

 29%|██▉       | 287901/1000000 [7:17:38<15:31:48, 12.74it/s]

 29%|██▉       | 287951/1000000 [7:17:45<18:24:48, 10.74it/s]

 29%|██▉       | 288001/1000000 [7:17:58<29:01:35,  6.81it/s]

 29%|██▉       | 288051/1000000 [7:18:00<22:49:28,  8.66it/s]

 29%|██▉       | 288101/1000000 [7:18:04<20:27:23,  9.67it/s]

 29%|██▉       | 288201/1000000 [7:18:20<23:34:42,  8.39it/s]

 29%|██▉       | 288301/1000000 [7:18:25<19:29:09, 10.15it/s]

 29%|██▉       | 288351/1000000 [7:18:26<15:13:22, 12.99it/s]

 29%|██▉       | 288401/1000000 [7:18:33<19:29:15, 10.1

 30%|███       | 301451/1000000 [7:37:43<17:19:43, 11.20it/s]

 30%|███       | 301501/1000000 [7:37:43<12:23:27, 15.66it/s]

 30%|███       | 301551/1000000 [7:37:53<20:13:48,  9.59it/s]

 30%|███       | 301651/1000000 [7:37:57<16:34:25, 11.70it/s]

 30%|███       | 301701/1000000 [7:38:00<14:09:40, 13.70it/s]

 30%|███       | 301751/1000000 [7:38:04<15:27:53, 12.54it/s]

 30%|███       | 301801/1000000 [7:38:14<21:51:23,  8.87it/s]

 30%|███       | 301851/1000000 [7:38:16<18:06:47, 10.71it/s]

 30%|███       | 301901/1000000 [7:38:23<20:17:31,  9.56it/s]

 30%|███       | 301951/1000000 [7:38:24<15:48:50, 12.26it/s]

 30%|███       | 302001/1000000 [7:38:34<22:23:20,  8.66it/s]

 30%|███       | 302051/1000000 [7:38:37<19:49:07,  9.78it/s]

 30%|███       | 302101/1000000 [7:38:40<17:03:09, 11.37it/s]

 30%|███       | 302151/1000000 [7:38:44<16:37:08, 11.66it/s]

 30%|███       | 302201/1000000 [7:38:50<18:04:26, 10.72it/s]

 30%|███       | 302251/1000000 [7:38:51<14:06:50, 13.7

 32%|███▏      | 315201/1000000 [7:59:02<19:41:55,  9.66it/s]

 32%|███▏      | 315301/1000000 [7:59:13<19:57:45,  9.53it/s]

 32%|███▏      | 315401/1000000 [7:59:16<15:48:38, 12.03it/s]

 32%|███▏      | 315401/1000000 [7:59:31<15:48:38, 12.03it/s]

 32%|███▏      | 315451/1000000 [7:59:32<29:55:18,  6.36it/s]

 32%|███▏      | 315551/1000000 [7:59:33<21:23:36,  8.89it/s]

 32%|███▏      | 315601/1000000 [7:59:37<19:44:13,  9.63it/s]

 32%|███▏      | 315651/1000000 [7:59:39<15:31:04, 12.25it/s]

 32%|███▏      | 315701/1000000 [7:59:45<17:37:34, 10.78it/s]

 32%|███▏      | 315751/1000000 [7:59:46<13:49:00, 13.76it/s]

 32%|███▏      | 315801/1000000 [7:59:51<15:09:30, 12.54it/s]

 32%|███▏      | 315851/1000000 [7:59:53<13:18:55, 14.27it/s]

 32%|███▏      | 315901/1000000 [8:00:01<18:02:58, 10.53it/s]

 32%|███▏      | 316001/1000000 [8:00:06<15:44:48, 12.07it/s]

 32%|███▏      | 316051/1000000 [8:00:14<19:22:35,  9.80it/s]

 32%|███▏      | 316101/1000000 [8:00:23<24:30:40,  7.7

...

 94%|█████████▎| 936351/1000000 [23:35:33<2:06:05,  8.41it/s]

 94%|█████████▎| 936401/1000000 [23:35:38<2:03:12,  8.60it/s]

 94%|█████████▎| 936451/1000000 [23:35:40<1:35:32, 11.09it/s]

 94%|█████████▎| 936501/1000000 [23:35:46<1:45:06, 10.07it/s]

 94%|█████████▎| 936551/1000000 [23:35:52<1:52:51,  9.37it/s]

 94%|█████████▎| 936651/1000000 [23:35:59<1:40:42, 10.48it/s]

 94%|█████████▎| 936701/1000000 [23:36:00<1:17:47, 13.56it/s]

 94%|█████████▎| 936751/1000000 [23:36:06<1:33:41, 11.25it/s]

 94%|█████████▎| 936801/1000000 [23:36:14<1:56:28,  9.04it/s]

 94%|█████████▎| 936851/1000000 [23:36:20<1:59:04,  8.84it/s]

 94%|█████████▎| 936901/1000000 [23:36:25<1:51:40,  9.42it/s]

 94%|█████████▎| 936951/1000000 [23:36:34<2:13:53,  7.85it/s]

 94%|█████████▎| 937051/1000000 [23:36:40<1:54:53,  9.13it/s]

 94%|█████████▎| 937101/1000000 [23:36:42<1:29:43, 11.68it/s]

 94%|█████████▎| 937151/1000000 [23:36:47<1:36:30, 10.85it/s]

 94%|█████████▎| 937201/1000000 [23:36:51<1:32:36, 11.3

 95%|█████████▌| 950801/1000000 [23:58:44<1:09:29, 11.80it/s]

 95%|█████████▌| 950851/1000000 [23:58:45<2:06:35,  6.47it/s]

 95%|█████████▌| 950951/1000000 [23:58:45<1:28:53,  9.20it/s]

 95%|█████████▌| 951001/1000000 [23:58:51<1:27:42,  9.31it/s]

 95%|█████████▌| 951051/1000000 [23:58:52<1:06:50, 12.21it/s]

 95%|█████████▌| 951101/1000000 [23:58:58<1:18:10, 10.43it/s]

 95%|█████████▌| 951151/1000000 [23:58:59<57:41, 14.11it/s]  

 95%|█████████▌| 951201/1000000 [23:59:05<1:09:46, 11.66it/s]

 95%|█████████▌| 951251/1000000 [23:59:14<1:33:56,  8.65it/s]

 95%|█████████▌| 951301/1000000 [23:59:19<1:27:37,  9.26it/s]

 95%|█████████▌| 951351/1000000 [23:59:26<1:37:01,  8.36it/s]

 95%|█████████▌| 951451/1000000 [23:59:33<1:25:47,  9.43it/s]

 95%|█████████▌| 951551/1000000 [23:59:41<1:19:38, 10.14it/s]

 95%|█████████▌| 951601/1000000 [23:59:42<58:27, 13.80it/s]  

 95%|█████████▌| 951651/1000000 [23:59:51<1:25:53,  9.38it/s]

 95%|█████████▌| 951751/1000000 [23:59:56<1:11:31, 11.2

 96%|█████████▋| 964601/1000000 [24:19:49<1:03:06,  9.35it/s]

 96%|█████████▋| 964701/1000000 [24:20:01<1:05:34,  8.97it/s]

 96%|█████████▋| 964801/1000000 [24:20:04<51:28, 11.40it/s]  

 96%|█████████▋| 964851/1000000 [24:20:06<43:08, 13.58it/s]

 96%|█████████▋| 964901/1000000 [24:20:14<57:26, 10.18it/s]

 96%|█████████▋| 964951/1000000 [24:20:19<59:07,  9.88it/s]

 97%|█████████▋| 965001/1000000 [24:20:26<1:05:22,  8.92it/s]

 97%|█████████▋| 965101/1000000 [24:20:35<1:00:40,  9.59it/s]

 97%|█████████▋| 965201/1000000 [24:20:39<50:18, 11.53it/s]  

 97%|█████████▋| 965251/1000000 [24:20:43<46:34, 12.44it/s]

 97%|█████████▋| 965301/1000000 [24:20:48<51:21, 11.26it/s]

 97%|█████████▋| 965351/1000000 [24:20:54<55:34, 10.39it/s]

 97%|█████████▋| 965451/1000000 [24:21:02<52:59, 10.87it/s]

 97%|█████████▋| 965551/1000000 [24:21:13<55:22, 10.37it/s]

 97%|█████████▋| 965651/1000000 [24:21:23<57:20,  9.98it/s]

 97%|█████████▋| 965701/1000000 [24:21:25<44:07, 12.96it/s]

 97%|███████

 98%|█████████▊| 978551/1000000 [24:40:58<26:21, 13.56it/s]

 98%|█████████▊| 978601/1000000 [24:41:03<29:34, 12.06it/s]

 98%|█████████▊| 978651/1000000 [24:41:05<24:32, 14.50it/s]

 98%|█████████▊| 978701/1000000 [24:41:14<36:35,  9.70it/s]

 98%|█████████▊| 978751/1000000 [24:41:23<44:40,  7.93it/s]

 98%|█████████▊| 978801/1000000 [24:41:29<44:01,  8.02it/s]

 98%|█████████▊| 978851/1000000 [24:41:37<47:08,  7.48it/s]

 98%|█████████▊| 978901/1000000 [24:41:38<36:26,  9.65it/s]

 98%|█████████▊| 978951/1000000 [24:41:45<39:49,  8.81it/s]

 98%|█████████▊| 979001/1000000 [24:41:49<35:51,  9.76it/s]

 98%|█████████▊| 979051/1000000 [24:42:00<48:13,  7.24it/s]

 98%|█████████▊| 979151/1000000 [24:42:07<40:37,  8.55it/s]

 98%|█████████▊| 979201/1000000 [24:42:14<43:38,  7.94it/s]

 98%|█████████▊| 979251/1000000 [24:42:17<35:54,  9.63it/s]

 98%|█████████▊| 979301/1000000 [24:42:21<34:36,  9.97it/s]

 98%|█████████▊| 979351/1000000 [24:42:23<27:57, 12.31it/s]

 98%|█████████▊| 979351/

 99%|█████████▉| 992751/1000000 [25:02:43<12:45,  9.48it/s]

 99%|█████████▉| 992801/1000000 [25:02:45<10:21, 11.59it/s]

 99%|█████████▉| 992851/1000000 [25:02:52<11:51, 10.05it/s]

 99%|█████████▉| 992901/1000000 [25:02:54<09:52, 11.99it/s]

 99%|█████████▉| 992951/1000000 [25:02:58<09:29, 12.37it/s]

 99%|█████████▉| 993001/1000000 [25:03:02<09:47, 11.91it/s]

 99%|█████████▉| 993051/1000000 [25:03:05<08:54, 13.01it/s]

 99%|█████████▉| 993101/1000000 [25:03:21<17:25,  6.60it/s]

 99%|█████████▉| 993201/1000000 [25:03:23<12:23,  9.15it/s]

 99%|█████████▉| 993251/1000000 [25:03:30<13:36,  8.27it/s]

 99%|█████████▉| 993301/1000000 [25:03:32<10:32, 10.59it/s]

 99%|█████████▉| 993351/1000000 [25:03:38<11:38,  9.52it/s]

 99%|█████████▉| 993401/1000000 [25:03:40<09:18, 11.82it/s]

 99%|█████████▉| 993451/1000000 [25:03:45<09:54, 11.02it/s]

 99%|█████████▉| 993501/1000000 [25:03:50<10:07, 10.70it/s]

 99%|█████████▉| 993551/1000000 [25:03:54<09:21, 11.48it/s]

 99%|█████████▉| 993551/

In [24]:
len(scores_list)

1000000

In [29]:
errors = []
for score in tqdm(scores_list):
    if 'score' not in score['damaging']:
        if 'type' not in score['damaging']['error']:
            error_type = str(score['damaging']['error'])
        else:
            error_type = score['damaging']['error']['type']
        errors.append(error_type)




  0%|          | 0/1000000 [00:00<?, ?it/s]


 12%|█▏        | 124334/1000000 [00:00<00:00, 1243329.33it/s]


 24%|██▍       | 242029/1000000 [00:00<00:00, 1222640.97it/s]


 35%|███▌      | 350342/1000000 [00:00<00:00, 1177153.92it/s]


 46%|████▌     | 460668/1000000 [00:00<00:00, 1153965.49it/s]


 57%|█████▋    | 572855/1000000 [00:00<00:00, 1144144.33it/s]


 67%|██████▋   | 666668/1000000 [00:00<00:00, 998523.91it/s] 


 76%|███████▌  | 755434/1000000 [00:00<00:00, 851687.10it/s]


 84%|████████▎ | 836594/1000000 [00:00<00:00, 785001.91it/s]


100%|██████████| 1000000/1000000 [00:01<00:00, 994803.39it/s][A


In [31]:
len(errors)

5208

In [30]:
Counter(errors).most_common(50)

[('TextDeleted', 4068),
 ('RevisionNotFound', 546),
 ('TimeoutError', 196),
 ('CommentDeleted', 166),
 ('{\'code\': \'internal server error\', \'message\': \'Traceback (most recent call last):\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/connectionpool.py", line 421, in _make_request\\n    six.raise_from(e, None)\\n  File "<string>", line 3, in raise_from\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/connectionpool.py", line 416, in _make_request\\n    httplib_response = conn.getresponse()\\n  File "/usr/lib/python3.5/http/client.py", line 1198, in getresponse\\n    response.begin()\\n  File "/usr/lib/python3.5/http/client.py", line 297, in begin\\n    version, status, reason = self._read_status()\\n  File "/usr/lib/python3.5/http/client.py", line 258, in _read_status\\n    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")\\n  File "/usr/lib/python3.5/socket.py", line 576, in readinto\\n    return self._sock.recv

In [5]:
print(eval('{\'code\': \'internal server error\', \'message\': \'Traceback (most recent call last):\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/response.py", line 425, in _error_catcher\\n    yield\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/response.py", line 755, in read_chunked\\n    chunk = self._handle_chunk(amt)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/response.py", line 699, in _handle_chunk\\n    value = self._fp._safe_read(amt)\\n  File "/usr/lib/python3.5/http/client.py", line 607, in _safe_read\\n    chunk = self.fp.read(min(amt, MAXAMOUNT))\\n  File "/usr/lib/python3.5/socket.py", line 576, in readinto\\n    return self._sock.recv_into(b)\\n  File "/usr/lib/python3.5/ssl.py", line 937, in recv_into\\n    return self.read(nbytes, buffer)\\n  File "/usr/lib/python3.5/ssl.py", line 799, in read\\n    return self._sslobj.read(len, buffer)\\n  File "/usr/lib/python3.5/ssl.py", line 583, in read\\n    v = self._sslobj.read(len, buffer)\\nsocket.timeout: The read operation timed out\\n\\nDuring handling of the above exception, another exception occurred:\\n\\nTraceback (most recent call last):\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/requests/models.py", line 750, in generate\\n    for chunk in self.raw.stream(chunk_size, decode_content=True):\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/response.py", line 560, in stream\\n    for line in self.read_chunked(amt, decode_content=decode_content):\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/response.py", line 781, in read_chunked\\n    self._original_response.close()\\n  File "/usr/lib/python3.5/contextlib.py", line 77, in __exit__\\n    self.gen.throw(type, value, traceback)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/response.py", line 430, in _error_catcher\\n    raise ReadTimeoutError(self._pool, None, "Read timed out.")\\nurllib3.exceptions.ReadTimeoutError: HTTPSConnectionPool(host=\\\'en.wikipedia.org\\\', port=443): Read timed out.\\n\\nDuring handling of the above exception, another exception occurred:\\n\\nTraceback (most recent call last):\\n  File "./ores/wsgi/routes/v3/util.py", line 105, in process_score_request\\n    score_response = scoring_system.score(score_request)\\n  File "./ores/scoring_systems/scoring_system.py", line 60, in score\\n    response = self._score(request)\\n  File "./ores/scoring_systems/celery_queue.py", line 195, in _score\\n    return super()._score(*args, **kwargs)\\n  File "./ores/scoring_systems/scoring_system.py", line 105, in _score\\n    request, missing_model_set_revs)\\n  File "./ores/scoring_systems/scoring_system.py", line 152, in _extract_root_caches\\n    model_set, rev_ids, injection_caches=request.injection_caches)\\n  File "./ores/scoring_context.py", line 177, in extract_root_dependency_caches\\n    for rev_id, (error, values) in zip(rev_ids, error_root_vals):\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/revscoring/extractors/api/extractor.py", line 153, in _extract_many\\n    rvprop=rvprop)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/revscoring/extractors/api/extractor.py", line 233, in get_rev_doc_map\\n    return {rd[\\\'revid\\\']: rd for rd in rev_docs}\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/revscoring/extractors/api/extractor.py", line 233, in <dictcomp>\\n    return {rd[\\\'revid\\\']: rd for rd in rev_docs}\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/revscoring/extractors/api/extractor.py", line 244, in query_revisions_by_revids\\n    **params)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/mwapi/session.py", line 309, in get\\n    continuation=continuation)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/mwapi/session.py", line 171, in request\\n    files=files)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/mwapi/session.py", line 116, in _request\\n    doc = resp.json()\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/requests/models.py", line 897, in json\\n    return complexjson.loads(self.text, **kwargs)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/requests/models.py", line 852, in text\\n    if not self.content:\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/requests/models.py", line 828, in content\\n    self._content = b\\\'\\\'.join(self.iter_content(CONTENT_CHUNK_SIZE)) or b\\\'\\\'\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/requests/models.py", line 757, in generate\\n    raise ConnectionError(e)\\nrequests.exceptions.ConnectionError: HTTPSConnectionPool(host=\\\'en.wikipedia.org\\\', port=443): Read timed out.\\n\'}')['message'])

Traceback (most recent call last):
  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/response.py", line 425, in _error_catcher
    yield
  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/response.py", line 755, in read_chunked
    chunk = self._handle_chunk(amt)
  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/response.py", line 699, in _handle_chunk
    value = self._fp._safe_read(amt)
  File "/usr/lib/python3.5/http/client.py", line 607, in _safe_read
    chunk = self.fp.read(min(amt, MAXAMOUNT))
  File "/usr/lib/python3.5/socket.py", line 576, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.5/ssl.py", line 937, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.5/ssl.py", line 799, in read
    return self._sslobj.read(len, buffer)
  File "/usr/lib/python3.5/ssl.py", line 583, in read
    v = self._sslobj.read(len, buffer)
socket.timeout: The read operatio

In [6]:
print(eval('{\'code\': \'internal server error\', \'message\': \'Traceback (most recent call last):\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/connectionpool.py", line 421, in _make_request\\n    six.raise_from(e, None)\\n  File "<string>", line 3, in raise_from\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/connectionpool.py", line 416, in _make_request\\n    httplib_response = conn.getresponse()\\n  File "/usr/lib/python3.5/http/client.py", line 1198, in getresponse\\n    response.begin()\\n  File "/usr/lib/python3.5/http/client.py", line 297, in begin\\n    version, status, reason = self._read_status()\\n  File "/usr/lib/python3.5/http/client.py", line 258, in _read_status\\n    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")\\n  File "/usr/lib/python3.5/socket.py", line 576, in readinto\\n    return self._sock.recv_into(b)\\n  File "/usr/lib/python3.5/ssl.py", line 937, in recv_into\\n    return self.read(nbytes, buffer)\\n  File "/usr/lib/python3.5/ssl.py", line 799, in read\\n    return self._sslobj.read(len, buffer)\\n  File "/usr/lib/python3.5/ssl.py", line 583, in read\\n    v = self._sslobj.read(len, buffer)\\nsocket.timeout: The read operation timed out\\n\\nDuring handling of the above exception, another exception occurred:\\n\\nTraceback (most recent call last):\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/requests/adapters.py", line 449, in send\\n    timeout=timeout\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/connectionpool.py", line 720, in urlopen\\n    method, url, error=e, _pool=self, _stacktrace=sys.exc_info()[2]\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/util/retry.py", line 400, in increment\\n    raise six.reraise(type(error), error, _stacktrace)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/packages/six.py", line 735, in reraise\\n    raise value\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/connectionpool.py", line 672, in urlopen\\n    chunked=chunked,\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/connectionpool.py", line 423, in _make_request\\n    self._raise_timeout(err=e, url=url, timeout_value=read_timeout)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/connectionpool.py", line 331, in _raise_timeout\\n    self, url, "Read timed out. (read timeout=%s)" % timeout_value\\nurllib3.exceptions.ReadTimeoutError: HTTPSConnectionPool(host=\\\'en.wikipedia.org\\\', port=443): Read timed out. (read timeout=5.0)\\n\\nDuring handling of the above exception, another exception occurred:\\n\\nTraceback (most recent call last):\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/mwapi/session.py", line 101, in _request\\n    auth=auth)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/requests/sessions.py", line 533, in request\\n    resp = self.send(prep, **send_kwargs)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/requests/sessions.py", line 646, in send\\n    r = adapter.send(request, **kwargs)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/requests/adapters.py", line 529, in send\\n    raise ReadTimeout(e, request=request)\\nrequests.exceptions.ReadTimeout: HTTPSConnectionPool(host=\\\'en.wikipedia.org\\\', port=443): Read timed out. (read timeout=5.0)\\n\\nThe above exception was the direct cause of the following exception:\\n\\nTraceback (most recent call last):\\n  File "./ores/wsgi/routes/v3/util.py", line 105, in process_score_request\\n    score_response = scoring_system.score(score_request)\\n  File "./ores/scoring_systems/scoring_system.py", line 60, in score\\n    response = self._score(request)\\n  File "./ores/scoring_systems/celery_queue.py", line 195, in _score\\n    return super()._score(*args, **kwargs)\\n  File "./ores/scoring_systems/scoring_system.py", line 105, in _score\\n    request, missing_model_set_revs)\\n  File "./ores/scoring_systems/scoring_system.py", line 152, in _extract_root_caches\\n    model_set, rev_ids, injection_caches=request.injection_caches)\\n  File "./ores/scoring_context.py", line 177, in extract_root_dependency_caches\\n    for rev_id, (error, values) in zip(rev_ids, error_root_vals):\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/revscoring/extractors/api/extractor.py", line 126, in _extract_many\\n    rev_docs = self.get_rev_doc_map(revids_to_lookup, rvprop=rvprop)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/revscoring/extractors/api/extractor.py", line 233, in get_rev_doc_map\\n    return {rd[\\\'revid\\\']: rd for rd in rev_docs}\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/revscoring/extractors/api/extractor.py", line 233, in <dictcomp>\\n    return {rd[\\\'revid\\\']: rd for rd in rev_docs}\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/revscoring/extractors/api/extractor.py", line 244, in query_revisions_by_revids\\n    **params)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/mwapi/session.py", line 309, in get\\n    continuation=continuation)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/mwapi/session.py", line 171, in request\\n    files=files)\\n  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/mwapi/session.py", line 103, in _request\\n    raise TimeoutError(str(e)) from e\\nmwapi.errors.TimeoutError: HTTPSConnectionPool(host=\\\'en.wikipedia.org\\\', port=443): Read timed out. (read timeout=5.0)\\n\'}')['message'])

Traceback (most recent call last):
  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/srv/deployment/ores/deploy/venv/lib/python3.5/site-packages/urllib3/connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.5/http/client.py", line 1198, in getresponse
    response.begin()
  File "/usr/lib/python3.5/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.5/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.5/socket.py", line 576, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.5/ssl.py", line 937, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.5/ssl.py", line 799, in read
    return self._sslobj.r